In [4]:
from pymongo import MongoClient
from pymongo import ReturnDocument
import json, requests
from pprint import pprint

In [5]:
def getFaceAPICreds():
    credentials = json.load(open('../credentials.json'))
    faceCreds = credentials['cognitiveServices']['faceDetection']
    return faceCreds

def getFaceDetectURL(faceCreds):
    return(faceCreds['endPoint'] + '/detect')

def getFaceIdentifyURL(faceCreds):
    return(faceCreds['endPoint'] + '/identify')

def getPersonGroupURL(faceCreds):
    return(faceCreds['endPoint'] + '/persongroups')

def getFaceKey(faceCreds):
    return(faceCreds['key'])

In [6]:
def createHeaders(subscription_key, contentType):
    headers = {
    'Content-Type': 'application/' + contentType,
    'Ocp-Apim-Subscription-Key': subscription_key,
    }
    return(headers)

def createParams():
    params = {
    'returnFaceId': 'true',
    'returnFaceLandmarks': 'false',
    'returnFaceAttributes': 'age,gender,blur,exposure,noise'
    }
    return(params)

In [7]:
def getFaceDetails(res):
    faceId = res.json()[0]['faceId']
    noiseLevel = res.json()[0]['faceAttributes']['noise']['noiseLevel']
    blurLevel = res.json()[0]['faceAttributes']['blur']['blurLevel']
#     all attributes not necessary; for extra functionality later:
    return({
        "faceId" : faceId,
        "noiseLevel" : noiseLevel,
        "blurLevel" : blurLevel
    })

In [8]:
def getFaceId():
    faceCreds = getFaceAPICreds()
    key = getFaceKey(faceCreds)
    headers = createHeaders(key, 'octet-stream')
    params = createParams()
    url = getFaceDetectURL(faceCreds)
    body = open('./target.jpg', 'rb')
    response = requests.post(url=url,data=body,params=params,headers=headers)
    body.close()
    if( len(response.json()) == 0 ):
        return(False)
    faceID = getFaceDetails(response)['faceId']
    return(faceID)

In [33]:
def identifyPatient(personGroupId):
#     if(captureTargetImage()):
#         print("Image prepping for analysis")
#     else:
#         print("Image capture failed")
#         return(False)
    faceIds = []
    faceId = getFaceId()
    if(faceId == False):
        return("No faces detected!")
    faceIds.append(faceId)
    faceCreds = getFaceAPICreds()
    key = getFaceKey(faceCreds)
    url = getFaceIdentifyURL(faceCreds)
    headers = createHeaders(key, 'json')
    body = {
        "faceIds" : faceIds,
        "personGroupId" : personGroupId,
        "maxNumOfCandidatesReturned" : 1,
        "confidenceThreshold" : 0.7
    }
    req = requests.post(url=url, headers=headers, json=body)
    candidate = findPatientRecords(req.json())
    if(candidate == False):
        return("Face does not match any patients or Patient not registered")
    response = "Hello, " + candidate
    return(response)

In [34]:
def findPatient(personId):
    client = getClient()
    patientCollection = getPatients(client)
    dbResult = patientCollection.find_one({"personId" : personId})
    return(dbResult)

def getDBCreds():
    credentials = json.load(open('../credentials.json'))
    dbCreds = credentials['database']
    return dbCreds

def connectToDB(dbCreds):
    cosmosConnString = "mongodb://" + dbCreds['username'] + ":" + dbCreds['password'] + "@" + dbCreds['host'] + ":" + str(dbCreds['port']) + "/?ssl=true&replicaSet=globaldb"
    client = MongoClient(cosmosConnString)
    return client

def getClient():
    dbCreds = getDBCreds()
    client = connectToDB(dbCreds)
    return client

def getPatients(client):
    db = client["admin"]
    patientCollection = db["patients"]
    return patientCollection


In [35]:
def findPatientRecords(response):
    if(len(response[0]['candidates']) == 0):
        print("No recognized candidates")
        return(False)
    personId = response[0]['candidates'][0]['personId']
    result = findPatient(personId)
    pprint(result)
    if(result):
        result = result['name']
        return(result)
    else:
        print("Unable to find patient")
        return(False)

In [38]:
def addPersonFace(file, personGroupId, personId):
    faceCreds = getFaceAPICreds()
    key = getFaceKey(faceCreds)
    headers = createHeaders(key, 'octet-stream')
    url = getPersonGroupURL(faceCreds)
    fullURL = url + '/' + personGroupId + '/persons/' + personId + '/persistedFaces'
    body = file.read_bytes()
    req = requests.post(url=fullURL, data=body, headers=headers)
    print("Response: " + str(req.status_code) + "; Persisted Face ID: " + req.json()['persistedFaceId'])
    if(req.status_code != 200):
        print("Error: " + pprint(req.json()))
    else:
        print("Success")
        pprint(req.json())
    return(req)